In [1]:
import pandas as pd 
import numpy as np

In [2]:
data = pd.read_csv("output-onlinetools [MConverter.eu].csv")

In [3]:
data.head()

,Sn.,Application no,Product,Goods,State
0,1,1.0,Darjeeling Tea (word & logo),Agricultural,West Bengal
1,2,3.0,Aranmula Kannadi,Handicraft,Kerala
2,3,67.0,Molela Clay Work,Handicraft,Rajasthan
3,4,68.0,Kathputlis of Rajasthan,Handicraft,Rajasthan
4,5,97.0,Leather Toys of Indore,Handicraft,Madhya Pradesh


In [4]:
data.shape

(211, 5)

In [5]:
data = data.drop(columns=['Sn.'])

In [6]:
data.head()

,Application no,Product,Goods,State
0,1.0,Darjeeling Tea (word & logo),Agricultural,West Bengal
1,3.0,Aranmula Kannadi,Handicraft,Kerala
2,67.0,Molela Clay Work,Handicraft,Rajasthan
3,68.0,Kathputlis of Rajasthan,Handicraft,Rajasthan
4,97.0,Leather Toys of Indore,Handicraft,Madhya Pradesh


In [7]:
data.isnull().sum()

Application no    50
Product           32
Goods             54
State             55
dtype: int64

In [8]:
data.dropna(inplace=True)

In [9]:
data.shape

(130, 4)

In [10]:
data.head()

,Application no,Product,Goods,State
0,1.0,Darjeeling Tea (word & logo),Agricultural,West Bengal
1,3.0,Aranmula Kannadi,Handicraft,Kerala
2,67.0,Molela Clay Work,Handicraft,Rajasthan
3,68.0,Kathputlis of Rajasthan,Handicraft,Rajasthan
4,97.0,Leather Toys of Indore,Handicraft,Madhya Pradesh


In [11]:
data['tags'] = data['Product']+data['Goods']

In [12]:
data.head()

,Application no,Product,Goods,State,tags
0,1.0,Darjeeling Tea (word & logo),Agricultural,West Bengal,Darjeeling Tea (word & logo) Agricultural
1,3.0,Aranmula Kannadi,Handicraft,Kerala,Aranmula Kannadi Handicraft
2,67.0,Molela Clay Work,Handicraft,Rajasthan,Molela Clay Work Handicraft
3,68.0,Kathputlis of Rajasthan,Handicraft,Rajasthan,Kathputlis of Rajasthan Handicraft
4,97.0,Leather Toys of Indore,Handicraft,Madhya Pradesh,Leather Toys of Indore Handicraft


In [13]:
data['tags'][0]

'Darjeeling Tea (word & logo) Agricultural '

In [14]:
data['tags'] = data['tags'].apply(lambda x:"".join(x))

In [15]:
data['tags'].head(1)

0    Darjeeling Tea (word & logo) Agricultural 
Name: tags, dtype: object

In [16]:
import nltk

In [17]:
from nltk.stem.porter import PorterStemmer

In [18]:
ps = PorterStemmer()

In [19]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [20]:
data['tags'] = data['tags'].apply(lambda x:x.lower())
data['State'] = data['State'].apply(lambda x:x.lower())

In [21]:
data.head(1)

,Application no,Product,Goods,State,tags
0,1.0,Darjeeling Tea (word & logo),Agricultural,west bengal,darjeeling tea (word & logo) agricultural


In [22]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [23]:
vector = cv.fit_transform(data['tags']).toarray()

In [24]:
vector

array([[0, 0, 1, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [25]:
data['tags'] = data['tags'].apply(stem)

In [26]:
data['tags'][0]

'darjeel tea (word & logo) agricultur'

In [27]:
from sklearn.metrics.pairwise import cosine_similarity

In [28]:
similarity = cosine_similarity(vector)

In [29]:
def recommend(product_name, data, similarity):
        product_index = data[data['Product'] == product_name].index[0]
        distances = similarity[product_index]
        product_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
        print("Recommended products:")
        for i in product_list:
            print("THE PRODUCTS:-",data.iloc[i[0]]['Product'])
            print("STATE:-",data.iloc[i[0]]['State'])



In [30]:
example_product = data['Product'].iloc[0] 

In [31]:
recommend(example_product,data,similarity)

Recommended products:
THE PRODUCTS:- Kangra Tea 
STATE:- himachal pradesh 
THE PRODUCTS:- Pokkali Rice 
STATE:- kerala 
THE PRODUCTS:- Naga Mircha 
STATE:- nagaland 
THE PRODUCTS:- Nilgiri (Orthodox) 
STATE:- tamil nadu 
THE PRODUCTS:- Assam (Orthodox) 
STATE:- assam 


In [32]:
def recommend2(State_name,data,similarity):
    State_index = data[data['State']==State_name].index[0]
    distances = similarity[State_index]
    State_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    print('RECOMMENDED STATE')
    for i in State_list:
        print("THE PRODUCTS:-",data.iloc[i[0]]['Product'])
        print("THE STATE:-",data.iloc[i[0]]['State'])

In [33]:
example_state = data['State'].iloc[0]
print(example_state)

west bengal 


In [34]:
recommend2(example_state,data,similarity)

RECOMMENDED STATE
THE PRODUCTS:- Kangra Tea 
THE STATE:- himachal pradesh 
THE PRODUCTS:- Pokkali Rice 
THE STATE:- kerala 
THE PRODUCTS:- Naga Mircha 
THE STATE:- nagaland 
THE PRODUCTS:- Nilgiri (Orthodox) 
THE STATE:- tamil nadu 
THE PRODUCTS:- Assam (Orthodox) 
THE STATE:- assam 


In [35]:
def recommend3(State):
      state_data = data[data['State'] == State]
      print("THE PRODUCTS:")
      for product in state_data['Product'].head(5):
            print(product)
      print("THE GOODS:")
      for Goods in state_data['Goods'].head(5):
            print(Goods)

In [36]:
example_state2 = data['State'].iloc[23]

In [37]:
recommend3(example_state2)

THE PRODUCTS:
Kani Shawl 
Kashmir Pashmina 
Kashmir Sozani Craft 
THE GOODS:
Handicraft 
Handicraft 
Handicraft 


In [38]:
unique_states = data['State'].dropna().str.strip().unique()


In [39]:
unique_states_series = pd.Series(unique_states)

In [40]:
print(unique_states_series)

0                  west bengal
1                       kerala
2                    rajasthan
3               madhya pradesh
4                      gujarat
5                   tamil nadu
6              jammu & kashmir
7             himachal pradesh
8                    karnataka
9                       odisha
10                 maharashtra
11                   telangana
12                    nagaland
13                       assam
14               uttar pradesh
15                         goa
16              andhra pradesh
17                        peru
18                      france
19    united states of america
20              united kingdom
21                       italy
22                       bihar
23                       india
24                 chattisgarh
dtype: object


In [41]:
state_to_select = input("Enter the name of the state you want to select: ").strip()
print(state_to_select)

odisha


In [42]:
def state_recommend(state_to_select):
    columns_to_drop = ['tags','Application no'] 
    selected_state = data[data['State'].str.strip() == state_to_select]
    selected_state = selected_state.drop(columns=columns_to_drop)
    return selected_state

In [43]:
data['Goods'] = data['Goods'].apply(lambda x:x.lower())

In [44]:
state_recommend(state_to_select)

,Product,Goods,State
19,Pipli Applique Work,handicraft,odisha
116,Khandua Saree and Fabrics,handicraft,odisha
136,Orissa Ikat,handicraft,odisha
156,Kotpad Handloom fabric,handicraft,odisha
203,Konark Stone carving,handicraft,odisha
204,Orissa Pattachitra,handicraft,odisha


In [45]:
Goods_to_select = input("Enter your selected Goods:").strip()
print(Goods_to_select)

handicraft


In [46]:

def Goods_recommend(Goods_to_select):
    columns_to_drop = ['tags','Application no']
    selected_Goods = data[data['Goods'].str.strip()==Goods_to_select]
    selected_Goods = selected_Goods.drop(columns=columns_to_drop)
    return selected_Goods[1:7]

In [47]:
Goods_recommend(Goods_to_select)

,Product,Goods,State
2,Molela Clay Work,handicraft,rajasthan
3,Kathputlis of Rajasthan,handicraft,rajasthan
4,Leather Toys of Indore,handicraft,madhya pradesh
8,Sankheda Furniture,handicraft,gujarat
9,Agates of Cambay,handicraft,gujarat
12,Salem Fabric,handicraft,tamil nadu


In [48]:
def filter(Goods_to_select,state_to_select):
    columns_to_drop = ['tags','Application no']
    selected_Goods2 = data[data['Goods'].str.strip()==Goods_to_select]
    selected_Goods2 = selected_Goods2.drop(columns=columns_to_drop)
    selected_state2 = data[data['State'].str.strip()==state_to_select]
    selected_state2 = selected_state2.drop(columns=columns_to_drop)
    filter = selected_state2.merge(selected_Goods2,on=['State','Goods','Product'])
    return filter
    

In [49]:
filter(Goods_to_select,state_to_select)

,Product,Goods,State
0,Pipli Applique Work,handicraft,odisha
1,Khandua Saree and Fabrics,handicraft,odisha
2,Orissa Ikat,handicraft,odisha
3,Kotpad Handloom fabric,handicraft,odisha
4,Konark Stone carving,handicraft,odisha
5,Orissa Pattachitra,handicraft,odisha
